# CMOR example with prototype CMIP7 tables/CVs

This code is a demonstration of the use of CMOR with prototype CMIP7 CVs and tables. This will require a conda environment in which CMOR 3.10.0 has been installed

Please note that file and directory names required for CMIP7 have not been confirmed yet, this is just a limited demonstration.

In [1]:
%time
import cmor
import numpy 
import json

CPU times: user 2 μs, sys: 1e+03 ns, total: 3 μs
Wall time: 5.72 μs


The following is adapted from the code at https://github.com/PCMDI/cmor/blob/main/Test/test_cmor_CMIP7.py

Construct the input json file for CMOR


In [2]:
%time
DATASET_INFO = {
    "_AXIS_ENTRY_FILE": "tables/CMIP7_coordinate.json",
    "_FORMULA_VAR_FILE": "tables/CMIP7_formula_terms.json",
    "_cmip7_option": 1,
    "_controlled_vocabulary_file": "test/CMIP7-CV_for-cmor.json",  # SEPARATE TO MIP TABLES FOR TESTING ONLY
    "activity_id": "CMIP",
    "branch_method": "standard",
    "branch_time_in_child": 30.0,
    "branch_time_in_parent": 10800.0,
    "calendar": "360_day",
    "cv_version": "7.0.0.0",
    "drs_specs": "MIP-DRS7",
    "experiment_id": "historical",
    "forcing_index": "f3",
    "grid": "N96",
    "grid_label": "gn",
    "initialization_index": "i1",
    "institution_id": "PCMDI",
    "license_id": "CC BY 4.0",
    "nominal_resolution": "250 km",
    "outpath": ".",
    "parent_mip_era": "CMIP7",
    "parent_time_units": "days since 1850-01-01",
    "parent_activity_id": "CMIP",
    "parent_source_id": "PCMDI-test-1-0",
    "parent_experiment_id": "piControl",
    "parent_variant_label": "r1i1p1f3",
    "physics_index": "p1",
    "realization_index": "r9",
    "source_id": "PCMDI-test-1-0",
    "source_type": "AOGCM CHEM BGC",
    "tracking_prefix": "hdl:21.14100",
    "host_collection": "CMIP7",
    "frequency": "day",
    "region": "glb",
    "archive_id": "WCRP",
    "mip_era": "CMIP7",
}

with open('input.json', 'w') as fh:
    json.dump(DATASET_INFO, fh, indent=2)

CPU times: user 2 μs, sys: 1 μs, total: 3 μs
Wall time: 5.48 μs


In [3]:
%time
cmor.setup(inpath="tables", netcdf_file_action=cmor.CMOR_REPLACE)

cmor.dataset_json('input.json')

tos = numpy.array([27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27,
                    27, 27, 27, 27
                    ])
tos.shape = (2, 3, 4)
lat = numpy.array([10, 20, 30])
lat_bnds = numpy.array([5, 15, 25, 35])
lon = numpy.array([0, 90, 180, 270])
lon_bnds = numpy.array([-45, 45,
                        135,
                        225,
                        315
                        ])
time = numpy.array([15.5, 16.5])
time_bnds = numpy.array([15, 16, 17])

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 5.48 μs


In [4]:
%time
cmor.load_table("CMIP7_ocean.json")

CPU times: user 2 μs, sys: 1e+03 ns, total: 3 μs
Wall time: 5.01 μs


0

In [5]:
%time
cmorlat = cmor.axis("latitude",
                    coord_vals=lat,
                    cell_bounds=lat_bnds,
                    units="degrees_north")
cmorlon = cmor.axis("longitude",
                    coord_vals=lon,
                    cell_bounds=lon_bnds,
                    units="degrees_east")
cmortime = cmor.axis("time",
                    coord_vals=time,
                    cell_bounds=time_bnds,
                    units="days since 2018")
axes = [cmortime, cmorlat, cmorlon]

CPU times: user 2 μs, sys: 1 μs, total: 3 μs
Wall time: 5.01 μs


In [6]:
cmortos = cmor.variable("tos_tavg-u-hxy-sea", "degC", axes)

In [7]:
cmor.write(cmortos, tos)
filename = cmor.close(cmortos, file_name=True)

Locate generated file

In [8]:
!find MIP-DRS7 -type f -name *.nc

MIP-DRS7/CMIP7/CMIP/PCMDI/PCMDI-test-1-0/historical/r9i1p1f3/glb/day/tos/tavg-u-hxy-sea/gn/v20251016/tos_tavg-u-hxy-sea_day_glb_gn_PCMDI-test-1-0_historical_r9i1p1f3_20180116-20180117.nc


Dump header of generated file

In [9]:
!ncdump -h MIP-DRS7/CMIP7/CMIP/PCMDI/PCMDI-test-1-0/historical/r9i1p1f3/glb/day/tos/tavg-u-hxy-sea/gn/*/tos_tavg-u-hxy-sea_day_glb_gn_PCMDI-test-1-0_historical_r9i1p1f3_20180116-20180117.nc

netcdf tos_tavg-u-hxy-sea_day_glb_gn_PCMDI-test-1-0_historical_r9i1p1f3_20180116-20180117 {
dimensions:
	time = UNLIMITED ; // (2 currently)
	lat = 3 ;
	lon = 4 ;
	bnds = 2 ;
variables:
	double time(time) ;
		time:bounds = "time_bnds" ;
		time:units = "days since 2018" ;
		time:calendar = "360_day" ;
		time:axis = "T" ;
		time:long_name = "Time Intervals" ;
		time:standard_name = "time" ;
	double time_bnds(time, bnds) ;
	double lat(lat) ;
		lat:bounds = "lat_bnds" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
		lat:long_name = "Latitude" ;
		lat:standard_name = "latitude" ;
	double lat_bnds(lat, bnds) ;
	double lon(lon) ;
		lon:bounds = "lon_bnds" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
		lon:long_name = "Longitude" ;
		lon:standard_name = "longitude" ;
	double lon_bnds(lon, bnds) ;
	float tos(time, lat, lon) ;
		tos:standard_name = "sea_surface_temperature" ;
		tos:long_name = "Sea Surface Temperature" ;
		tos:units = "degC" ;
		tos:cell_methods = "area: mean where 

In [10]:
# clean up
! rm -rf MIP-DRS7
! rm -rf input.json

rm: cannot remove 'MIP-DRS7/CMIP7/CMIP/PCMDI/PCMDI-test-1-0/historical/r9i1p1f3/glb/day/tos/tavg-u-hxy-sea/gn/v20251016/.nfs000000000300845e00107e47': Device or resource busy
